In [1]:
import numpy as np
import pandas as pd
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.preprocessing import OneHotEncoder
print 'march_import'

march_import


In [2]:
# utils
## chinese decode 
def cn(s):
#     return s.decode('utf8')
    return s
## 字符串类型转换为类别的数字编码
dict_short_url  = {'init':0} #和短地址一样，统一记录使用过的int
dict_short_url_index = {0: 'init'}
def getShortUrl(s):
    # 未记录，则添加记录
    if dict_short_url.has_key(s) == False:
        dict_short_url[s] = max(dict_short_url.values())+1
        dict_short_url_index[dict_short_url[s]] = s
    return dict_short_url[s]

In [4]:
# read data
# trainData means rawdata, X_train means cross_validation data
# trainData = pd.merge(pd.read_csv(cn('E:\\work\\联通+旅游\\data\\1\\train_x.csv'),encoding="gb2312"), pd.read_csv(cn('E:\\work\\联通+旅游\\data\\1\\train_y.csv'),encoding="gb2312"), on=cn('用户标识'))
# testData = pd.read_csv(cn('E:\\work\\联通+旅游\\data\\1\\test_x.csv'),encoding="gb2312")
trainData = pd.merge(pd.read_csv('/data/topic1/train_x.csv'), pd.read_csv('/data/topic1/train_y.csv'), on='用户标识')
testData = pd.read_csv('/data/topic1/test_x.csv')
print 'march_load_data'

march_load_data


In [ ]:
# 读入外部特征
## 训练集
trainData = pd.merge(trainData, pd.read_csv('/data/topic1/feature_countapppv.csv'), on='用户标识')
trainData = pd.merge(trainData, pd.read_csv('/data/topic1/feature_countapp.csv'), on='用户标识')
trainData = pd.merge(trainData, pd.read_csv('/data/topic1/feature_countoutprovince.csv'), on='用户标识')
trainData = pd.merge(trainData, pd.read_csv('/data/topic1/feature_countwebpv.csv'), on='用户标识')
trainData = pd.merge(trainData, pd.read_csv('/data/topic1/feature_countweb.csv'), on='用户标识')
trainData = pd.merge(trainData, pd.read_csv('/data/topic1/feature_countgroupticket'), on='用户标识')
trainData = pd.merge(trainData, pd.read_csv('/data/topic1/feature_countgrouptraffic'), on='用户标识')
trainData = pd.merge(trainData, pd.read_csv('/data/topic1/feature_countgroupfff'), on='用户标识')
trainData = pd.merge(trainData, pd.read_csv('/data/topic1/feature_countgrouphotel'), on='用户标识')
trainData = pd.merge(trainData, pd.read_csv('/data/topic1/feature_countgroupweather'), on='用户标识')
trainData = pd.merge(trainData, pd.read_csv('/data/topic1/feature_cluster.csv'), on='用户标识')
trainData = pd.merge(trainData, pd.read_csv('/data/topic1/feature_outprovince.csv'), on='用户标识')
trainData = pd.merge(trainData, pd.read_csv('/data/topic1/feature_topweb.csv'), on='用户标识')
trainData = pd.merge(trainData, pd.read_csv('/data/topic1/feature_webcluster.csv'), on='用户标识')
trainData = pd.merge(trainData, pd.read_csv('/data/topic1/feature_maxgroup.csv'), on='用户标识')
## 测试集
testData = pd.merge(testData, pd.read_csv('/data/topic1/feature_test_countapppv.csv'), on='用户标识')
testData = pd.merge(testData, pd.read_csv('/data/topic1/feature_test_countapp.csv'), on='用户标识')
testData = pd.merge(testData, pd.read_csv('/data/topic1/feature_test_countoutprovince.csv'), on='用户标识')
testData = pd.merge(testData, pd.read_csv('/data/topic1/feature_test_countwebpv.csv'), on='用户标识')
testData = pd.merge(testData, pd.read_csv('/data/topic1/feature_test_countweb.csv'), on='用户标识')
testData = pd.merge(testData, pd.read_csv('/data/topic1/feature_test_countgroupticket'), on='用户标识')
testData = pd.merge(testData, pd.read_csv('/data/topic1/feature_test_countgrouptraffic'), on='用户标识')
testData = pd.merge(testData, pd.read_csv('/data/topic1/feature_test_countgroupfff'), on='用户标识')
testData = pd.merge(testData, pd.read_csv('/data/topic1/feature_test_countgrouphotel'), on='用户标识')
testData = pd.merge(testData, pd.read_csv('/data/topic1/feature_test_countgroupweather'), on='用户标识')
testData = pd.merge(testData, pd.read_csv('/data/topic1/feature_test_cluster.csv'), on='用户标识')
testData = pd.merge(testData, pd.read_csv('/data/topic1/feature_test_outprovince.csv'), on='用户标识')
testData = pd.merge(testData, pd.read_csv('/data/topic1/feature_test_topweb.csv'), on='用户标识')
testData = pd.merge(testData, pd.read_csv('/data/topic1/feature_test_webcluster.csv'), on='用户标识')
testData = pd.merge(testData, pd.read_csv('/data/topic1/feature_test_maxgroup.csv'), on='用户标识')

In [5]:
# 生成特征
## 训练集
trainData['phone_brand'] = trainData[cn('手机品牌')].apply(getShortUrl)
trainData['phone_model_number'] = trainData[cn('手机终端型号')].apply(getShortUrl)
trainData['if_cross_province'] = trainData[cn('是否有跨省行为')].apply(getShortUrl)
trainData['if_cross_conuntry'] = trainData[cn('是否有出境行为')].apply(getShortUrl)
trainData['topweb'] = trainData['topweb'].apply(getShortUrl)
## 测试集
testData['phone_brand'] = testData[cn('手机品牌')].apply(getShortUrl)
testData['phone_model_number'] = testData[cn('手机终端型号')].apply(getShortUrl)
testData['if_cross_province'] = testData[cn('是否有跨省行为')].apply(getShortUrl)
testData['if_cross_conuntry'] = testData[cn('是否有出境行为')].apply(getShortUrl)
testData['topweb'] = testData['topweb'].apply(getShortUrl)

In [6]:
trainData.drop([cn('手机品牌'), cn('是否有跨省行为'), cn('是否有出境行为'), cn('漫入省份'), cn('漫出省份'), cn('手机终端型号')], axis=1, inplace=True)
testData.drop([cn('手机品牌'), cn('是否有跨省行为'), cn('是否有出境行为'), cn('漫入省份'), cn('漫出省份'), cn('手机终端型号')], axis=1, inplace=True)

## 模型训练

In [8]:
# 修改列名， lgb不支持中文列名
dict_new_columns = {}
for i in trainData.drop([cn('用户标识'),cn('是否去过迪士尼')],axis=1).columns:
    dict_new_columns[i] = str(getShortUrl(i))
trainData.rename(columns=dict_new_columns, inplace = True)
testData.rename(columns=dict_new_columns, inplace = True)

In [9]:
# 声明类别变量,
temp_categorical_feature = [cn('性别'),cn('大致消费水平'),'phone_brand','phone_model_number','if_cross_province','if_cross_conuntry',
                           'hainan','shanghai','neimenggu','xizang','gansu','henan','cluster','webcluster', 'topweb','maxgroup']
list_categorical_feature = [str(dict_short_url[item]) for item in temp_categorical_feature]

In [ ]:
# # 一次训练
# print 'march_init_model'
# train_data = lgb.Dataset(trainData.drop([cn('用户标识'),cn('是否去过迪士尼')],axis=1).values, label=trainData[cn('是否去过迪士尼')], feature_name=list(trainData.drop([cn('用户标识'),cn('是否去过迪士尼')],axis=1).columns), categorical_feature=list_categorical_feature)
# param = {'application': 'binary', 'boosting': 'gbdt', 'num_leaves':127, 'num_trees':1000, 'objective':'binary', 'min_child_samples':400, 'max_bin': 250, 'max_depth':-1, 
#          'feature_fraction':1, 'metric':'auc','subsample':0.9, 'lambda_l1':1.5, 'lambda_l2':20, 'min_data_in_bin':100,
#         'bagging_fraction':1, 'bagging_freq':1, 'learning_rate':0.025}
# estimators = lgb.train(param,train_data,num_boost_round=150)
# ypred = estimators.predict(testData.drop([cn('用户标识')],axis=1).values)
# print 'march_model_done'

In [19]:
# 多次训练取平均
print 'march_init_model_avg'
list_ypred = []
for i in range(3):
    this_round_data = trainData.sample(frac=0.9)
    train_data = lgb.Dataset(this_round_data.drop([cn('用户标识'),cn('是否去过迪士尼')],axis=1).values, label=this_round_data[cn('是否去过迪士尼')], feature_name=list(this_round_data.drop([cn('用户标识'),cn('是否去过迪士尼')],axis=1).columns), categorical_feature=list_categorical_feature)
    param = {'application': 'binary', 'boosting': 'dart', 'num_leaves':127, 'num_trees':1000, 'objective':'binary', 'min_child_samples':400, 'max_bin': 250, 'max_depth':-1, 
         'feature_fraction':1, 'metric':'auc','subsample':0.9, 'lambda_l1':1.5, 'lambda_l2':20, 'min_data_in_bin':100,
        'bagging_fraction':1, 'bagging_freq':1, 'learning_rate':0.05}
    estimators = lgb.train(param,train_data,num_boost_round=486)
    ypred = estimators.predict(testData.drop([cn('用户标识')],axis=1).values)
    list_ypred.append(ypred)
ypred =  np.asarray(list_ypred).mean(axis=0)
print 'march_model_done'

march_init_model_avg
march_model_done


In [22]:
# 输出csv结果
output = pd.DataFrame({'IMEI': testData[cn('用户标识')].values,'SCORE':ypred})
# output.to_csv(cn('E:\\work\\联通+旅游\\data\\1\\result.csv'), index=False, float_format='%.5f')
output.to_csv('/data/topic1/result_lgb1.csv', index=False, float_format='%.5f')